In [ ]:
%pip install ultralytics

import torch
from ultralytics import YOLO

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


In [ ]:
# Path to your data.yaml
data_path = '../data/data.yaml'

# Show the data.yaml content
with open(data_path, 'r') as f:
    print(f.read())


In [ ]:
model = YOLO('../models/yolov8n.pt')
results = model.train(
    data='../data/data.yaml',
    epochs=100,
    patience=10,
    imgsz=640,
    batch=8,
    name='wardrobe_yolov8n_macos',
    project='../outputs'
)

# Run validation on the best model
metrics = model.val()
print(metrics)


In [ ]:
%pip install opencv-python

import cv2
from ultralytics import YOLO

# Load your trained model
model = YOLO('../outputs/models/yolov8n_finetuned.pt')

# Open webcam 1 (change the index as needed)
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Run YOLO prediction
    results = model.predict(frame, show=True)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


2025-05-02 13:57:22.070 Python[13618:3785542] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.



0: 384x640 (no detections), 36.9ms
Speed: 2.5ms preprocess, 36.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 42.8ms
Speed: 3.3ms preprocess, 42.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 61.3ms
Speed: 4.0ms preprocess, 61.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 53.9ms
Speed: 3.3ms preprocess, 53.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 49.7ms
Speed: 3.4ms preprocess, 49.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 63.4ms
Speed: 4.3ms preprocess, 63.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 39.6ms
Speed: 2.3ms preprocess, 39.6ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 43.7ms
Speed: 2.1ms preprocess, 43.7ms i

KeyboardInterrupt: 

: 